# Lesson 8.3: State Management and Conditional Edges

---

In Lesson 8.2, we built a simple sequential graph with LangGraph. However, the true power of LangGraph lies in its ability to manage complex states and create non-linear control flows. This lesson will delve into how data is passed and updated within the graph, as well as how to use **Conditional Edges** to create logical branches based on conditions.

## 1. Concept of State in LangGraph: How data is passed and updated

The **State** is the heart of every LangGraph graph. It is a single, mutable data object that is passed from one Node to another throughout the graph's execution.

* **Data Passing:** Each **Node** in the graph receives the current copy of the state as input.
* **Data Updates:** After performing its logic, a Node will return a **dictionary** containing updates to the state. LangGraph will automatically **merge** these updates into the overall state.
* **Merge Mechanism:** This is a crucial aspect. When you define your `State Schema` using `TypedDict` (which will be covered in more detail later), you can specify how new values for each field will be combined with existing values.
    * **Default Overwrite:** If nothing is specified, the new value will overwrite the old one.
    * **`operator.add`:** For lists, `operator.add` (or `list.append`) is often used to append new elements to the existing list, rather than completely replacing the list. This is very useful for `chat_history`.
    * **Custom Merging:** You can define custom merge functions for more complex data types.




---

## 2. Using TypedDict to Define the Graph's State Structure

To define the structure of the state clearly and type-safely, LangGraph recommends using `TypedDict` from Python's `typing` library.

* **Syntax:** You create a class that inherits from `TypedDict` and declare the fields along with their data types.
* **`Annotated` and `operator.add`:** To specify a merge mechanism for a particular field (e.g., `chat_history`), you use `Annotated` along with a merge function.

In [ ]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import operator

# Define a more complex state structure
class AgentState(TypedDict):
    # Chat history: List of messages, will be appended
    chat_history: Annotated[List[BaseMessage], operator.add]
    # Query classification: String (e.g., "SEARCH_NEEDED", "DIRECT_ANSWER")
    query_classification: str
    # Search results: String containing results from the search tool
    search_results: str
    # Other temporary variables
    temp_data: Annotated[List[str], operator.add]
    # Flag to control loops (e.g., True/False)
    should_continue: bool

**Explanation:**
* `chat_history` will have new messages appended.
* `query_classification` and `search_results` will be overwritten each time a Node returns a new value for them (since no special merge mechanism is specified).
* `temp_data` will also be appended.
* `should_continue` will be overwritten.

Defining the state clearly helps you easily track data and ensure that Nodes only access and update the parts of the data they need.


---

## 3. Conditional Edges: Creating branches in the graph based on results or conditions from a Node

In real-world Agent applications, the processing flow is rarely just sequential. Agents need the ability to make decisions and branch based on conditions. **Conditional Edges** are LangGraph's mechanism for achieving this.

* **Concept:** Instead of going from Node A to Node B in a fixed manner, a Node can return a value (typically a string) that LangGraph uses to decide the next Node.
* **How it Works:**
    1.  A Node performs processing and returns a string (e.g., "search", "answer", "loop").
    2.  LangGraph uses this string value to look up in a defined mapping.
    3.  This mapping indicates the next Node corresponding to that string value.
    4.  If the Node returns `END`, the graph will terminate.




---

## 4. Using `add_conditional_edges` to route flow based on logic

The `add_conditional_edges` method is how you define conditional branches in LangGraph.

* **Syntax:**
    ```python
    graph_builder.add_conditional_edges(
        start_node: str,
        condition: Callable[[State], str],
        path_map: Dict[str, str]
    )
    ```
    * `start_node`: The name of the Node from which you want to create conditional edges.
    * `condition`: A Python function that takes the current graph state and returns a string (the name of the target Node) or `END`. This function contains the decision-making logic.
    * `path_map`: A dictionary mapping the return values from the `condition` function to the actual target Node names in the graph.

* **Example of `condition` function:**
    ```python
    from langgraph.graph import END

    def route_next_step(state: AgentState) -> str:
        """
        This function decides the next step based on 'query_classification' in the state.
        """
        if state["query_classification"] == "SEARCH_NEEDED":
            return "perform_search_node"
        elif state["query_classification"] == "DIRECT_ANSWER":
            return "generate_final_answer_node"
        else:
            return END # End if no condition matches (or error handling)
    ```

* **Example of `add_conditional_edges`:**
    ```python
    workflow.add_conditional_edges(
        "classify_query_node", # Node from which we branch
        route_next_step,      # Function that decides the path
        {
            "SEARCH_NEEDED": "perform_search_node",
            "DIRECT_ANSWER": "generate_final_answer_node",
            END: END # If route_next_step function returns END, the graph ends
        }
    )
    ```


---

## 5. Practical Example: Building a Graph with Branching Logic

We will build a simple chatbot graph that can branch:
* If the question requires external information, it will go through a search Node.
* If the question can be answered directly, it will skip the search Node and answer immediately.

**Preparation:**
* Ensure you have `langchain-openai`, `google-search-results` installed.
* Set the `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables.

In [ ]:
# Cài đặt thư viện nếu chưa có
# pip install langchain-openai openai google-search-results

import os
from typing import TypedDict, Annotated, List, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
import operator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool

# Thiết lập biến môi trường cho khóa API của OpenAI và SerpAPI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"

# --- 1. Định nghĩa kiểu trạng thái cho đồ thị ---
class AgentState(TypedDict):
    chat_history: Annotated[List[BaseMessage], operator.add] # Lịch sử trò chuyện
    initial_query: str # Giữ câu hỏi gốc của người dùng
    query_classification: str # "SEARCH_NEEDED" hoặc "DIRECT_ANSWER"
    search_results: str # Kết quả từ công cụ tìm kiếm

# --- 2. Khởi tạo LLM và Tools ---
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Tool tìm kiếm
search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Hữu ích khi bạn cần tìm kiếm thông tin trên Google về các sự kiện hiện tại hoặc dữ liệu thực tế." # Useful when you need to search for information on Google about current events or factual data.
)

# --- 3. Định nghĩa các Node ---

# Node 1: Lấy đầu vào ban đầu
def initial_input_node(state: AgentState) -> Dict[str, Any]:
    """Node này lấy tin nhắn Human mới nhất và lưu vào initial_query."""
    print("\n--- Node: Initial Input (Lấy câu hỏi gốc) ---") # --- Node: Initial Input (Get original question) ---
    last_message = state["chat_history"][-1]
    return {"initial_query": last_message.content}

# Node 2: Phân loại truy vấn (LLM quyết định)
def classify_query_node(state: AgentState) -> Dict[str, Any]:
    """
    Node này sử dụng LLM để phân loại xem câu hỏi có cần tìm kiếm hay không.
    Trả về "SEARCH_NEEDED" hoặc "DIRECT_ANSWER".
    """\n
    print("--- Node: Classify Query (Phân loại câu hỏi) ---") # --- Node: Classify Query (Classify question) ---
    query = state["initial_query"]
    classification_prompt = ChatPromptTemplate.from_template(
        """Phân loại câu hỏi sau đây: "{query}".
        Trả lời "SEARCH_NEEDED" nếu câu hỏi cần tìm kiếm thông tin bên ngoài (ví dụ: dữ kiện, tin tức, thời tiết, số liệu cụ thể).
        Trả lời "DIRECT_ANSWER" nếu câu hỏi có thể được trả lời trực tiếp bằng kiến thức chung của bạn hoặc không cần tìm kiếm.
        Chỉ trả lời "SEARCH_NEEDED" hoặc "DIRECT_ANSWER".""" # Classify the following question: "{query}". Reply "SEARCH_NEEDED" if the question requires external information (e.g., facts, news, weather, specific data). Reply "DIRECT_ANSWER" if the question can be answered directly using your general knowledge or does not require a search. Only reply "SEARCH_NEEDED" or "DIRECT_ANSWER".
    )
    chain = classification_prompt | llm | StrOutputParser()
    classification = chain.invoke({"query": query}).strip().upper()
    print(f"  Phân loại: {classification}") #   Classification:
    return {"query_classification": classification}

# Node 3: Thực hiện tìm kiếm (nếu cần)
def perform_search_node(state: AgentState) -> Dict[str, Any]:
    """Node này thực hiện tìm kiếm bằng công cụ tìm kiếm."""
    print("--- Node: Perform Search (Thực hiện tìm kiếm) ---") # --- Node: Perform Search (Perform search) ---
    query = state["initial_query"]
    search_result = search_tool.run(query)
    print(f"  Kết quả tìm kiếm: {search_result[:100]}...") #   Search result:
    return {"search_results": search_result}

# Node 4: Tạo câu trả lời cuối cùng
def generate_final_answer_node(state: AgentState) -> Dict[str, Any]:
    """Node này tạo câu trả lời cuối cùng dựa trên câu hỏi và kết quả tìm kiếm (nếu có)."""
    print("--- Node: Generate Final Answer (Tạo câu trả lời cuối cùng) ---") # --- Node: Generate Final Answer (Generate final answer) ---
    query = state["initial_query"]
    search_results = state.get("search_results", "Không có thông tin tìm kiếm.") # Lấy kết quả tìm kiếm nếu có # No search information.
    
    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", "Bạn là một trợ lý hữu ích. Trả lời câu hỏi sau đây. Nếu có kết quả tìm kiếm, hãy sử dụng chúng. Nếu không, trả lời dựa trên kiến thức chung của bạn."), # You are a helpful assistant. Answer the following question. If search results are available, use them. Otherwise, answer based on your general knowledge.
        ("human", f"Câu hỏi: {query}\n\nKết quả tìm kiếm: {search_results}"), # Question: {query}\n\nSearch Results: {search_results}
    ])
    chain = answer_prompt | llm | StrOutputParser()
    final_answer = chain.invoke({"question": query, "search_results": search_results})
    
    # Thêm câu trả lời của AI vào chat_history
    return {"chat_history": [AIMessage(content=final_answer)]}

# --- 4. Định nghĩa hàm điều kiện cho Conditional Edge ---
def route_next_step(state: AgentState) -> str:
    """
    Hàm này quyết định Node tiếp theo dựa trên 'query_classification'.
    """
    if state["query_classification"] == "SEARCH_NEEDED":
        return "perform_search_node"
    elif state["query_classification"] == "DIRECT_ANSWER":
        return "generate_final_answer_node"
    else:
        # Trường hợp không mong muốn, có thể chuyển đến Node xử lý lỗi hoặc kết thúc
        print(f"  Phân loại không xác định: {state['query_classification']}. Kết thúc.") # Unknown classification: {state['query_classification']}. Ending.
        return END

# --- 5. Xây dựng đồ thị ---
workflow = StateGraph(AgentState)

# Thêm các Node
workflow.add_node("initial_input", initial_input_node)
workflow.add_node("classify_query", classify_query_node)
workflow.add_node("perform_search_node", perform_search_node)
workflow.add_node("generate_final_answer_node", generate_final_answer_node)

# Đặt điểm bắt đầu
workflow.set_entry_point("initial_input")

# Định nghĩa cạnh cố định từ initial_input đến classify_query
workflow.add_edge("initial_input", "classify_query")

# Định nghĩa cạnh có điều kiện từ classify_query
workflow.add_conditional_edges(
    "classify_query", # Node mà từ đó chúng ta rẽ nhánh
    route_next_step,  # Hàm quyết định đường đi
    {
        "SEARCH_NEEDED": "perform_search_node",    # Nếu "SEARCH_NEEDED", đi đến perform_search_node
        "DIRECT_ANSWER": "generate_final_answer_node", # Nếu "DIRECT_ANSWER", đi đến generate_final_answer_node
        END: END # Nếu hàm route_next_step trả về END, thì đồ thị kết thúc
    }
)

# Định nghĩa cạnh cố định từ perform_search_node đến generate_final_answer_node
workflow.add_edge("perform_search_node", "generate_final_answer_node")

# Đặt điểm kết thúc
workflow.set_finish_point("generate_final_answer_node")

# Biên dịch đồ thị
app = workflow.compile()

print("\n--- Bắt đầu thực hành đồ thị có logic phân nhánh ---") # --- Starting practical: graph with branching logic ---

# --- Tình huống 1: Câu hỏi cần tìm kiếm ---
print("\n--- Tình huống 1: Câu hỏi cần tìm kiếm ---") # --- Scenario 1: Question requires search ---
initial_state_1 = {"chat_history": [HumanMessage(content="Ai là tổng thống hiện tại của Hoa Kỳ?")]} # Who is the current president of the United States?
final_state_1 = app.invoke(initial_state_1)
print(f"\nPhản hồi cuối cùng:") # Final response:
for message in final_state_1["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

# --- Tình huống 2: Câu hỏi có thể trả lời trực tiếp ---
print("\n--- Tình huống 2: Câu hỏi có thể trả lời trực tiếp ---") # --- Scenario 2: Question can be answered directly ---
initial_state_2 = {"chat_history": [HumanMessage(content="Mặt trời có màu gì?")]} # What color is the sun?
final_state_2 = app.invoke(initial_state_2)
print(f"\nPhản hồi cuối cùng:") # Final response:
for message in final_state_2["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

print("\n--- Kết thúc thực hành ---") # --- End of practical ---
